In [5]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [6]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
boardname    esp8266sens2

Sent 5 lines (108 bytes) to config.txt.


In [2]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
boardname    esp8266sens2

Sent 5 lines (115 bytes) to config.txt.


In [3]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.


In [219]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [4]:
%sendtofile main.py

import time
from machine import Pin, PWM, I2C

fconfig = dict(x.split()  for x in open("config.txt"))
pinled = Pin(int(fconfig["pinled"]), Pin.OUT)
for i in range(10):
    pinled.value(i%2)
    time.sleep_ms(300)
    
i2c = machine.I2C(scl=machine.Pin(4), sda=machine.Pin(5))
print(list(map(hex, i2c.scan())))

from mqtt_as import config, MQTTClient
import uasyncio as asyncio
from BME280_funcs import bme280init, readBME280


config['server'] = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]

boardname = fconfig["boardname"].encode()    

tbme280A = boardname+b'/bme280/A'
tbme280B = boardname+b'/bme280/B'

client = MQTTClient(config)

async def ledtask():
    while True:
        pinled.value(0)
        await asyncio.sleep_ms(20 if client.isconnected() else 600)
        pinled.value(1)
        await asyncio.sleep_ms(2000)
        
async def bme280task(addr, topic):
    btoinit = True
    while True:
        try:
            if btoinit:
                bme280init(i2c, addr)
                btoinit = False
                await asyncio.sleep_ms(100)
            meas = readBME280(addr)
            print(topic, meas)
            payload = " ".join(map(str, meas))
            await client.publish(topic, payload)
        except OSError as e:
            print("bme280task", e)
            btoinit = True
        await asyncio.sleep_ms(500)
        
async def connectingtask():
    while True:
        try:
            await client.connect()
            break
        except OSError as e:
            print([e])
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(boardname+b'/ip', ipnumber, retain=True, qos=1)
        
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(0x76, boardname+b'/bme280/A'))
aloop.create_task(bme280task(0x77, boardname+b'/bme280/B'))
aloop.create_task(ledtask())
aloop.create_task(connectingtask())
aloop.run_forever()



Sent 71 lines (2018 bytes) to main.py.
